# Win Loss Ratio and League Points - Europe

In [13]:
import numpy as np
import pandas as pd
import requests as re
import time

In [14]:
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.81 Safari/537.36 Edg/104.0.1293.47",
    "Accept-Language": "en-CA,en-US;q=0.9,en;q=0.8",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": "RGAPI-59d23d98-22f7-47b2-be13-5851816103a6"
}

In [15]:
KR = pd.read_csv('../../data/first_model/raw/KR_all_data_358.csv')
print(KR.shape)
KR.head()

(358, 51)


,SERVER,MATCH_ID,BSUM1_ID,BTOP_CHAMP_ID,BSUM2_ID,BJGL_CHAMP_ID,BSUM3_ID,BMID_CHAMP_ID,BSUM4_ID,BBOT_CHAMP_ID,...,FIRST_TOWER,BTOWER_KILLS,RTOWER_KILLS,FIRST_BARON,BBARON_KILLS,RBARON_KILLS,FIRST_INHIB,BINHIB_KILLS,RINHIB_KILLS,WINNER
0,KOREA,KR_6057844675,sbDHeqqUvLDhNA3yYnS3t93fG4d9doqldZjOkb3rwrybN_...,59,3PV_W4RY09TzgEQfBZNdSlsHFwxjwbjzIwXsawLuoSVYyp...,60,C8JeB0UVt7XBWdBC1EsTOMUecwXQ7M2s3aT8BiB6GCyoE4E,777,agy7tVJmthI4QFrgcC_GjBbhdd0juuBeCIIRwH-sS0AM1ek,157,...,RED,1,10,RED,0,1,RED,0,1,RED
1,KOREA,KR_6055710166,TOvKujDuw_rTCUebjSXidH8OJf_IoReR-YNkEDRhF-xCoOA,266,8K1yx07pCws-gzQxIzqyuoyVtEGbaUk35YQDtbEyoFgIA2Y,234,cTfhhY3u5vrB4LTFhUaJgVggwi1IKy14mnQRaL4lYT-SBQ,103,neTBID6mtg2VghrIdOtPVxRrBRJ4elf07mPURSA9hfz9bPc,15,...,BLUE,5,0,NONE,0,0,NONE,0,0,BLUE
2,KOREA,KR_6055326646,vw7N9YLAgtt3OEuzBHDzq7gQjIcdhK-crpvGNdbm7co2dck,29,EV23s0Fe5ptC8cyk1148besP5YnAtguSS9golPxZRwvwEyo,62,5C7Ab5bDdvKhppo5D5JoCiGkuW98oNYJLrjZPfo5cqpG20M,888,aFwzyFYDcqUvZje-bKywj8NPVlMxGj-MfCahYynTXvYVysQ,887,...,BLUE,5,0,NONE,0,0,NONE,0,0,BLUE
3,KOREA,KR_6053723720,5A4WJpFsCzkRoVDHzNAVO6bre5LRlLSlu5bDjX2oBOs9qKE,113,MDqGtPmXx3f2LMeKFTuWuI53bD4usXQCxyua869BLnh8Wx...,80,KgSuGOGxEJikfizSiVtI5v9QcqtN2aPrEmWAyP3Rj9bkuB0,711,T9cTTfK038tIME3U2xPWTaO4Vl20PhEfKnp9JQSobN8OCrk,523,...,RED,6,10,BLUE,1,0,RED,1,2,RED
4,KOREA,KR_6053619349,q3wZkwddVzV3gqDEZuE7l65m8U0j40aFprEdroCXXRyFXi0,14,Tx5p5q4af3CF8ckXbeZCZk4-RD8WepIX3KU1NzF5BS5swQ...,64,FR8ABgAmWaIHci98kpniqL3iddOz_W2zUv3gfV_GjpT74vGk,711,lhTGXwclX7jr-nYbvC1dwC4LaHBJhddgLzqhzO-GeMu62T...,221,...,BLUE,8,3,BLUE,1,0,BLUE,1,0,BLUE


In [16]:
KR1 = KR[:100]
KR2 = KR[100:200]
KR3 = KR[200:300]
KR4 = KR[300:]

In [17]:
'''
This function uses the champion mastery api to get the champion mastery for a specific champion and summoner.

Region can be: euw1, na1, kr

Summoner input can be:
-  BSUM1 - BSUM5
-  RSUM1 - RSUM5

SummonerID input can be:
-  BSUM1_ID - BSUM5_ID
-  RSUM1_ID - RSUM5_ID
'''

def summoner_stats(df, server, summoner, summonerID):

    stats_df = pd.DataFrame()
    wins_column = summoner + '_WINS'
    losses_column = summoner + '_LOSSES'
    league_points_column = summoner + '_LEAGUE_POINTS'

    wins_list = []
    losses_list = []
    league_points_list = []

    for i in range(len(df)):
        summoner_id = df.iloc[i][summonerID]
        try: 
            url = 'https://'+ server + '.api.riotgames.com/lol/league/v4/entries/by-summoner/' + summoner_id
            res = re.get(url, headers=headers).json()

            wins = res[0]['wins']
            wins_list.append(wins)

            losses = res[0]['losses']
            losses_list.append(losses)

            leaguePoints = res[0]['leaguePoints']
            league_points_list.append(leaguePoints)
        
        except:
            wins_list.append(np.nan)
            losses_list.append(np.nan)
            league_points_list.append(np.nan)
            pass

    stats_df[wins_column] = wins_list
    stats_df[losses_column] = losses_list
    stats_df[league_points_column] = league_points_list
        

    return stats_df

## Blue Team

### SUMMONER 1 - TOP

In [18]:
BTOP1 = summoner_stats(KR1, 'kr', 'BTOP', 'BSUM1_ID')
time.sleep(120)
BTOP2 = summoner_stats(KR2, 'kr', 'BTOP', 'BSUM1_ID')
time.sleep(120)
BTOP3 = summoner_stats(KR3, 'kr', 'BTOP', 'BSUM1_ID')
time.sleep(120)
BTOP4 = summoner_stats(KR4, 'kr', 'BTOP', 'BSUM1_ID')
time.sleep(120)
BTOP_stats = pd.concat([BTOP1, BTOP2, BTOP3, BTOP4], axis=0)
print(BTOP_stats.shape)
BTOP_stats.head()

(358, 3)


,BTOP_WINS,BTOP_LOSSES,BTOP_LEAGUE_POINTS
0,743,641,1464
1,751,697,773
2,455,405,1016
3,200,135,1154
4,668,606,1166


In [45]:
BTOP_stats.isna().any()

BTOP_WINS             False
BTOP_LOSSES           False
BTOP_LEAGUE_POINTS    False
dtype: bool

### SUMMONER 2 - JGL

In [20]:
BJGL1 = summoner_stats(KR1, 'kr', 'BJGL', 'BSUM2_ID')
time.sleep(120)
BJGL2 = summoner_stats(KR2, 'kr', 'BJGL', 'BSUM2_ID')
time.sleep(120)
BJGL3 = summoner_stats(KR3, 'kr', 'BJGL', 'BSUM2_ID')
time.sleep(120)
BJGL4 = summoner_stats(KR4, 'kr', 'BJGL', 'BSUM2_ID')
time.sleep(120)
BJGL_stats = pd.concat([BJGL1, BJGL2, BJGL3, BJGL4], axis=0)
print(BJGL_stats.shape)
BJGL_stats.head()

(358, 3)


,BJGL_WINS,BJGL_LOSSES,BJGL_LEAGUE_POINTS
0,138,104,691
1,581,528,919
2,280,226,1031
3,668,587,1140
4,505,453,991


In [44]:
BJGL_stats.isna().any()

BJGL_WINS             False
BJGL_LOSSES           False
BJGL_LEAGUE_POINTS    False
dtype: bool

### SUMMONER 3 - MID

In [21]:
BMID1 = summoner_stats(KR1, 'kr', 'BMID', 'BSUM3_ID')
time.sleep(120)
BMID2 = summoner_stats(KR2, 'kr', 'BMID', 'BSUM3_ID')
time.sleep(120)
BMID3 = summoner_stats(KR3, 'kr', 'BMID', 'BSUM3_ID')
time.sleep(120)
BMID4 = summoner_stats(KR4, 'kr', 'BMID', 'BSUM3_ID')
time.sleep(120)
BMID_stats = pd.concat([BMID1, BMID2, BMID3, BMID4], axis=0)
print(BMID_stats.shape)
BMID_stats.head()

(358, 3)


,BMID_WINS,BMID_LOSSES,BMID_LEAGUE_POINTS
0,448,410,733
1,465,414,942
2,783,721,1121
3,772,708,1076
4,533,477,1023


In [43]:
BMID_stats.isna().any()

BMID_WINS             False
BMID_LOSSES           False
BMID_LEAGUE_POINTS    False
dtype: bool

### SUMMONER 4 - BOT

In [22]:
BBOT1 = summoner_stats(KR1, 'kr', 'BBOT', 'BSUM4_ID')
time.sleep(120)
BBOT2 = summoner_stats(KR2, 'kr', 'BBOT', 'BSUM4_ID')
time.sleep(120)
BBOT3 = summoner_stats(KR3, 'kr', 'BBOT', 'BSUM4_ID')
time.sleep(120)
BBOT4 = summoner_stats(KR4, 'kr', 'BBOT', 'BSUM4_ID')
time.sleep(120)
BBOT_stats = pd.concat([BBOT1, BBOT2, BBOT3, BBOT4], axis=0)
print(BBOT_stats.shape)
BBOT_stats.head()

(358, 3)


,BBOT_WINS,BBOT_LOSSES,BBOT_LEAGUE_POINTS
0,444,373,1294
1,724,660,1073
2,403,344,1038
3,796,737,1013
4,663,607,891


In [42]:
BBOT_stats.isna().any()

BBOT_WINS             False
BBOT_LOSSES           False
BBOT_LEAGUE_POINTS    False
dtype: bool

### SUMMONER 5 - SUP

In [23]:
BSUP1 = summoner_stats(KR1, 'kr', 'BSUP', 'BSUM5_ID')
time.sleep(120)
BSUP2 = summoner_stats(KR2, 'kr', 'BSUP', 'BSUM5_ID')
time.sleep(120)
BSUP3 = summoner_stats(KR3, 'kr', 'BSUP', 'BSUM5_ID')
time.sleep(120)
BSUP4 = summoner_stats(KR4, 'kr', 'BSUP', 'BSUM5_ID')
time.sleep(120)
BSUP_stats = pd.concat([BSUP1, BSUP2, BSUP3, BSUP4], axis=0)
print(BSUP_stats.shape)
BSUP_stats.head()

(358, 3)


,BSUP_WINS,BSUP_LOSSES,BSUP_LEAGUE_POINTS
0,473,422,893
1,360,277,1018
2,465,414,942
3,381,325,1030
4,4,2,28


In [41]:
BSUP_stats.isna().any()

BSUP_WINS             False
BSUP_LOSSES           False
BSUP_LEAGUE_POINTS    False
dtype: bool

## Red Team

### Summoner 1 - TOP

In [24]:
RTOP1 = summoner_stats(KR1, 'kr', 'RTOP', 'RSUM1_ID')
time.sleep(120)
RTOP2 = summoner_stats(KR2, 'kr', 'RTOP', 'RSUM1_ID')
time.sleep(120)
RTOP3 = summoner_stats(KR3, 'kr', 'RTOP', 'RSUM1_ID')
time.sleep(120)
RTOP4 = summoner_stats(KR4, 'kr', 'RTOP', 'RSUM1_ID')
time.sleep(120)
RTOP_stats = pd.concat([RTOP1, RTOP2, RTOP3, RTOP4], axis=0)
print(RTOP_stats.shape)
RTOP_stats.head()

(358, 3)


,RTOP_WINS,RTOP_LOSSES,RTOP_LEAGUE_POINTS
0,425,359,1086
1,415,339,1196
2,577,531,901
3,20,18,100
4,512,457,815


In [40]:
RTOP_stats.isna().any()

RTOP_WINS             False
RTOP_LOSSES           False
RTOP_LEAGUE_POINTS    False
dtype: bool

### Summoner 2 - JGL

In [25]:
RJGL1 = summoner_stats(KR1, 'kr', 'RJGL', 'RSUM2_ID')
time.sleep(120)
RJGL2 = summoner_stats(KR2, 'kr', 'RJGL', 'RSUM2_ID')
time.sleep(120)
RJGL3 = summoner_stats(KR3, 'kr', 'RJGL', 'RSUM2_ID')
time.sleep(120)
RJGL4 = summoner_stats(KR4, 'kr', 'RJGL', 'RSUM2_ID')
time.sleep(120)
RJGL_stats = pd.concat([RJGL1, RJGL2, RJGL3, RJGL4], axis=0)
print(RJGL_stats.shape)
RJGL_stats.head()

(358, 3)


,RJGL_WINS,RJGL_LOSSES,RJGL_LEAGUE_POINTS
0,575,518,947
1,711,677,525
2,386,332,1071
3,744,641,1481
4,10,5,75


In [39]:
RJGL_stats.isna().any()

RJGL_WINS             False
RJGL_LOSSES           False
RJGL_LEAGUE_POINTS    False
dtype: bool

### Summoner 3 - MID

In [27]:
RMID1 = summoner_stats(KR1, 'kr', 'RMID', 'RSUM3_ID')
time.sleep(120)
RMID2 = summoner_stats(KR2, 'kr', 'RMID', 'RSUM3_ID')
time.sleep(120)
RMID3 = summoner_stats(KR3, 'kr', 'RMID', 'RSUM3_ID')
time.sleep(120)
RMID4 = summoner_stats(KR4, 'kr', 'RMID', 'RSUM3_ID')
time.sleep(120)
RMID_stats = pd.concat([RMID1, RMID2, RMID3, RMID4], axis=0)
print(RMID_stats.shape)
RMID_stats.head()

(358, 3)


,RMID_WINS,RMID_LOSSES,RMID_LEAGUE_POINTS
0,465.0,414.0,942.0
1,303.0,269.0,698.0
2,662.0,590.0,1059.0
3,465.0,414.0,942.0
4,465.0,414.0,942.0


In [38]:
RMID_stats.isna().any()

RMID_WINS             True
RMID_LOSSES           True
RMID_LEAGUE_POINTS    True
dtype: bool

### Summoner 4 - BOT

In [28]:
RBOT1 = summoner_stats(KR1, 'kr', 'RBOT', 'RSUM4_ID')
time.sleep(120)
RBOT2 = summoner_stats(KR2, 'kr', 'RBOT', 'RSUM4_ID')
time.sleep(120)
RBOT3 = summoner_stats(KR3, 'kr', 'RBOT', 'RSUM4_ID')
time.sleep(120)
RBOT4 = summoner_stats(KR4, 'kr', 'RBOT', 'RSUM4_ID')
time.sleep(120)
RBOT_stats = pd.concat([RBOT1, RBOT2, RBOT3, RBOT4], axis=0)
print(RBOT_stats.shape)
RBOT_stats.head()

(358, 3)


,RBOT_WINS,RBOT_LOSSES,RBOT_LEAGUE_POINTS
0,117.0,66.0,709.0
1,544.0,466.0,966.0
2,506.0,457.0,721.0
3,615.0,539.0,1038.0
4,204.0,143.0,754.0


In [37]:
RBOT_stats.isna().any()

RBOT_WINS             True
RBOT_LOSSES           True
RBOT_LEAGUE_POINTS    True
dtype: bool

### Summoner 5 - SUP

In [34]:
RSUP1 = summoner_stats(KR1, 'kr', 'RSUP', 'RSUM5_ID')
time.sleep(120)
RSUP2 = summoner_stats(KR2, 'kr', 'RSUP', 'RSUM5_ID')
time.sleep(120)
RSUP3 = summoner_stats(KR3, 'kr', 'RSUP', 'RSUM5_ID')
time.sleep(120)
RSUP4 = summoner_stats(KR4, 'kr', 'RSUP', 'RSUM5_ID')
time.sleep(120)
RSUP_stats = pd.concat([RSUP1, RSUP2, RSUP3, RSUP4], axis=0)
print(RSUP_stats.shape)
RSUP_stats.head()

(358, 3)


,RSUP_WINS,RSUP_LOSSES,RSUP_LEAGUE_POINTS
0,565,473,1103
1,392,336,673
2,538,482,1234
3,559,486,1069
4,259,235,624


In [36]:
RBOT_stats.isna().any()

RBOT_WINS             True
RBOT_LOSSES           True
RBOT_LEAGUE_POINTS    True
dtype: bool

### Combine into one DataFrame

In [30]:
KR_stats = pd.concat([BTOP_stats, BJGL_stats, BMID_stats, BBOT_stats, BSUP_stats, RTOP_stats, RJGL_stats, RMID_stats, RBOT_stats, RSUP_stats], axis=1)
print(KR_stats.shape)
KR_stats.head()

(358, 30)


,BTOP_WINS,BTOP_LOSSES,BTOP_LEAGUE_POINTS,BJGL_WINS,BJGL_LOSSES,BJGL_LEAGUE_POINTS,BMID_WINS,BMID_LOSSES,BMID_LEAGUE_POINTS,BBOT_WINS,...,RJGL_LEAGUE_POINTS,RMID_WINS,RMID_LOSSES,RMID_LEAGUE_POINTS,RBOT_WINS,RBOT_LOSSES,RBOT_LEAGUE_POINTS,RSUP_WINS,RSUP_LOSSES,RSUP_LEAGUE_POINTS
0,743,641,1464,138,104,691,448,410,733,444,...,947,465.0,414.0,942.0,117.0,66.0,709.0,565.0,473.0,1103.0
1,751,697,773,581,528,919,465,414,942,724,...,525,303.0,269.0,698.0,544.0,466.0,966.0,392.0,336.0,673.0
2,455,405,1016,280,226,1031,783,721,1121,403,...,1071,662.0,590.0,1059.0,506.0,457.0,721.0,538.0,482.0,1234.0
3,200,135,1154,668,587,1140,772,708,1076,796,...,1481,465.0,414.0,942.0,615.0,539.0,1038.0,559.0,486.0,1069.0
4,668,606,1166,505,453,991,533,477,1023,663,...,75,465.0,414.0,942.0,204.0,143.0,754.0,259.0,235.0,624.0


In [31]:
KR_stats.to_csv('KR_stats_358.csv', index=False)